## Введение 

В этом уроке будут рассмотрены: 
- Что такое вызов функции и случаи его использования 
- Как создать вызов функции с помощью OpenAI 
- Как интегрировать вызов функции в приложение 

## Цели обучения 

После завершения этого урока вы будете знать, как и понимать: 

- Цель использования вызова функции 
- Настройка вызова функции с использованием сервиса OpenAI 
- Проектирование эффективных вызовов функций для сценария использования вашего приложения


## Понимание вызовов функций

Для этого урока мы хотим создать функцию для нашего образовательного стартапа, которая позволит пользователям использовать чат-бота для поиска технических курсов. Мы будем рекомендовать курсы, соответствующие их уровню навыков, текущей роли и интересующей технологии.

Для этого мы будем использовать комбинацию:
 - `OpenAI` для создания чат-опыта для пользователя
 - `Microsoft Learn Catalog API` для помощи пользователям в поиске курсов на основе запроса пользователя
 - `Function Calling` для обработки запроса пользователя и отправки его в функцию для выполнения API-запроса.

Для начала давайте посмотрим, почему мы хотим использовать вызов функций в первую очередь:

print("Сообщения в следующем запросе:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # получить новый ответ от GPT, который может видеть ответ функции


print(second_response.choices[0].message)


### Почему вызов функций

Если вы прошли любой другой урок в этом курсе, вы, вероятно, понимаете мощь использования больших языковых моделей (LLM). Надеюсь, вы также видите некоторые их ограничения.

Вызов функций — это функция сервиса OpenAI, разработанная для решения следующих задач:

Несогласованное форматирование ответов:
- До появления вызова функций ответы большой языковой модели были неструктурированными и непоследовательными. Разработчикам приходилось писать сложный код валидации для обработки каждого варианта вывода.

Ограниченная интеграция с внешними данными:
- До этой функции было сложно включать данные из других частей приложения в контекст чата.

Стандартизируя форматы ответов и обеспечивая бесшовную интеграцию с внешними данными, вызов функций упрощает разработку и снижает необходимость дополнительной логики валидации.

Пользователи не могли получить ответы на вопросы вроде «Какая сейчас погода в Стокгольме?». Это связано с тем, что модели ограничены временем, на которое были обучены данные.

Рассмотрим пример ниже, иллюстрирующий эту проблему:

Допустим, мы хотим создать базу данных студентов, чтобы предлагать им подходящие курсы. Ниже приведены два описания студентов, которые очень похожи по содержащимся в них данным.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Мы хотим отправить это в LLM для анализа данных. Это позже можно будет использовать в нашем приложении для отправки на API или сохранения в базе данных.

Давайте создадим два идентичных запроса, в которых мы укажем LLM, какую информацию мы хотим получить:


Мы хотим отправить это в LLM, чтобы разобрать части, важные для нашего продукта. Таким образом, мы можем создать два идентичных запроса для инструктажа LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


После создания этих двух подсказок мы отправим их в LLM с помощью `openai.ChatCompletion`. Мы сохраняем подсказку в переменной `messages` и назначаем роль `user`. Это имитирует сообщение от пользователя, написанное для чат-бота.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Теперь мы можем отправить оба запроса в LLM и изучить полученный ответ.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Хотя подсказки одинаковы, а описания похожи, мы можем получить разные форматы свойства `Grades`.

Если запустить приведённую выше ячейку несколько раз, формат может быть `3.7` или `3.7 GPA`.

Это происходит потому, что LLM принимает неструктурированные данные в виде написанной подсказки и также возвращает неструктурированные данные. Нам нужен структурированный формат, чтобы знать, чего ожидать при сохранении или использовании этих данных.

Используя функциональный вызов, мы можем убедиться, что получаем обратно структурированные данные. При использовании функционального вызова LLM фактически не вызывает и не выполняет никаких функций. Вместо этого мы создаём структуру, которой LLM должен следовать в своих ответах. Затем мы используем эти структурированные ответы, чтобы знать, какую функцию запускать в наших приложениях.


![Диаграмма потока вызова функции](../../../../translated_images/Function-Flow.083875364af4f4bb.ru.png)


Затем мы можем взять то, что возвращается из функции, и отправить это обратно в LLM. LLM затем ответит, используя естественный язык, чтобы ответить на запрос пользователя.


### Сценарии использования вызовов функций

**Вызов внешних инструментов**  
Чат-боты отлично подходят для предоставления ответов на вопросы пользователей. С помощью вызова функций чат-боты могут использовать сообщения от пользователей для выполнения определённых задач. Например, студент может попросить чат-бота «Отправь письмо моему преподавателю с просьбой о дополнительной помощи по этому предмету». Это может привести к вызову функции `send_email(to: string, body: string)`.

**Создание запросов к API или базе данных**  
Пользователи могут находить информацию, используя естественный язык, который преобразуется в форматированный запрос или API-запрос. Примером может служить учитель, который спрашивает «Кто из студентов выполнил последнее задание», что может вызвать функцию с именем `get_completed(student_name: string, assignment: int, current_status: string)`.

**Создание структурированных данных**  
Пользователи могут взять блок текста или CSV и использовать LLM для извлечения из него важной информации. Например, студент может преобразовать статью из Википедии о мирных соглашениях для создания AI-флешкарт. Это можно сделать с помощью функции `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.


## 2. Создание вашего первого вызова функции

Процесс создания вызова функции включает 3 основных шага:  
1. Вызов API Chat Completions с списком ваших функций и сообщением пользователя  
2. Чтение ответа модели для выполнения действия, например, вызова функции или API  
3. Выполнение еще одного вызова API Chat Completions с ответом вашей функции, чтобы использовать эту информацию для создания ответа пользователю.


![Поток вызова функции](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.ru.png)


### Элементы вызова функции 

#### Ввод пользователя 

Первым шагом является создание сообщения пользователя. Оно может быть динамически присвоено путем получения значения из текстового поля или вы можете присвоить значение здесь. Если вы впервые работаете с API Chat Completions, нам нужно определить `role` и `content` сообщения. 

`role` может быть либо `system` (создание правил), `assistant` (модель) или `user` (конечный пользователь). Для вызова функции мы присвоим это значение как `user` и пример вопроса.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Создание функций.

Далее мы определим функцию и параметры этой функции. Здесь мы используем только одну функцию под названием `search_courses`, но вы можете создавать несколько функций.

**Важно**: Функции включаются в системное сообщение для LLM и учитываются в количестве доступных вам токенов.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Определения** 

Структура определения функции имеет несколько уровней, каждый со своими свойствами. Вот разбор вложенной структуры:

**Свойства функции верхнего уровня:**

`name` - Имя функции, которую мы хотим вызвать. 

`description` - Описание того, как работает функция. Здесь важно быть конкретным и ясным. 

`parameters` - Список значений и формата, которые вы хотите, чтобы модель использовала в своем ответе. 

**Свойства объекта параметров:**

`type` - Тип данных объекта параметров (обычно "object").

`properties` - Список конкретных значений, которые модель будет использовать в своем ответе. 

**Свойства отдельных параметров:**

`name` - Неявно определяется ключом свойства (например, "role", "product", "level").

`type` - Тип данных этого конкретного параметра (например, "string", "number", "boolean"). 

`description` - Описание конкретного параметра. 

**Необязательные свойства:**

`required` - Массив, перечисляющий параметры, которые обязательны для выполнения вызова функции. 


### Вызов функции  
После определения функции нам нужно включить её в вызов API Chat Completion. Мы делаем это, добавляя `functions` в запрос. В данном случае `functions=functions`.  

Также есть опция установить `function_call` в значение `auto`. Это означает, что мы позволим LLM самостоятельно решить, какую функцию вызвать, исходя из сообщения пользователя, вместо того чтобы назначать это вручную.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Теперь давайте посмотрим на ответ и увидим, как он отформатирован:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Вы можете видеть, что вызывается имя функции, и из сообщения пользователя LLM смог найти данные, соответствующие аргументам функции.


## 3.Интеграция вызовов функций в приложение. 


После того как мы протестировали отформатированный ответ от LLM, теперь мы можем интегрировать это в приложение. 

### Управление потоком 

Чтобы интегрировать это в наше приложение, давайте выполним следующие шаги: 

Сначала сделаем вызов к сервисам OpenAI и сохраним сообщение в переменную с именем `response_message`. 


In [ ]:
response_message = response.choices[0].message

Теперь мы определим функцию, которая будет вызывать API Microsoft Learn для получения списка курсов:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



В качестве лучшей практики мы затем проверим, хочет ли модель вызвать функцию. После этого мы создадим одну из доступных функций и сопоставим её с вызываемой функцией.  
Затем мы возьмём аргументы функции и сопоставим их с аргументами из LLM.

Наконец, мы добавим сообщение о вызове функции и значения, которые были возвращены сообщением `search_courses`. Это даёт LLM всю необходимую информацию для  
ответа пользователю на естественном языке.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Теперь мы отправим обновленное сообщение в LLM, чтобы получить ответ на естественном языке вместо ответа в формате JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Задача по программированию

Отличная работа! Чтобы продолжить изучение вызова функций OpenAI, вы можете создать: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Больше параметров функции, которые могут помочь учащимся найти больше курсов. Доступные параметры API можно найти здесь:  
 - Создайте еще один вызов функции, который принимает больше информации от учащегося, например, их родной язык  
 - Создайте обработку ошибок на случай, если вызов функции и/или вызов API не возвращает подходящих курсов


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, имейте в виду, что автоматический перевод может содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется обращаться к профессиональному переводу, выполненному человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
